# build metadata for tgt files

In [1]:
import pandas as pd
from functions import load_txt_as_lst

In [2]:
src_meta_path = "/scratch/craig.car/src_data/urn:cts:greekLit:tlg0001.tlg001.json"
src_meta = pd.read_json(src_meta_path)
src_meta

,loc,start,length
0,urn:cts:greekLit:tlg0001.tlg001:1.1,0,45
1,urn:cts:greekLit:tlg0001.tlg001:1.2,45,47
2,urn:cts:greekLit:tlg0001.tlg001:1.3,92,34
3,urn:cts:greekLit:tlg0001.tlg001:1.4,126,40
4,urn:cts:greekLit:tlg0001.tlg001:1.5,166,45
...,...,...,...
5829,urn:cts:greekLit:tlg0001.tlg001:4.1777,254548,43
5830,urn:cts:greekLit:tlg0001.tlg001:4.1778,254591,43
5831,urn:cts:greekLit:tlg0001.tlg001:4.1779,254634,43
5832,urn:cts:greekLit:tlg0001.tlg001:4.1780,254677,42


In [3]:
test_output_xml = "/scratch/craig.car/test.par"
test_par = load_txt_as_lst(test_output_xml)

In [4]:
test_par[1]

"book=1,chapter=1\t6\tCe que je tente aujourd’hui, plusieurs l’ont tenté avant moi; on a généralement loué leur entreprise. Chez quelques-uns même, ce n’est pas seulement au courageux effort que de bons juges ont applaudi. Où l’espoir de réussir toujours serait chimérique, ils ont eu du moins le bonheur de réussir souvent. La traduction des Choéphores, par M. Puech, que l’Université a trop tôt perdu, a eu pour elle des suffrages d’une grande autorité et est encore à bon droit estimée. M. Léon Halévy, après avoir déjà donné tant de preuves de son talent poétique, a tout récemment mérité d’être couronné par l’Académie française pour son troisième volume de la Grèce tragique, qui renferme une traduction des Euménides1. Sur un de nos théâtres de Paris, M. Alexandre Dumas a fait représenter une Orestie le 5 janvier 1856. Je devrais le compter, lui aussi, parmi ceux dont la rencontre était fort à craindre pour moi sur le chemin où ma témérité m’a engagé, s’il n’avait imité la célèbre trilogi

In [5]:
test_par[1].split("\t")

['book=1,chapter=1',
 '6',
 "Ce que je tente aujourd’hui, plusieurs l’ont tenté avant moi; on a généralement loué leur entreprise. Chez quelques-uns même, ce n’est pas seulement au courageux effort que de bons juges ont applaudi. Où l’espoir de réussir toujours serait chimérique, ils ont eu du moins le bonheur de réussir souvent. La traduction des Choéphores, par M. Puech, que l’Université a trop tôt perdu, a eu pour elle des suffrages d’une grande autorité et est encore à bon droit estimée. M. Léon Halévy, après avoir déjà donné tant de preuves de son talent poétique, a tout récemment mérité d’être couronné par l’Académie française pour son troisième volume de la Grèce tragique, qui renferme une traduction des Euménides1. Sur un de nos théâtres de Paris, M. Alexandre Dumas a fait représenter une Orestie le 5 janvier 1856. Je devrais le compter, lui aussi, parmi ceux dont la rencontre était fort à craindre pour moi sur le chemin où ma témérité m’a engagé, s’il n’avait imité la célèbre 

In [6]:
import spacy
import spacy_fastlang

In [7]:
spacy_fr_sm = spacy.load("fr_core_news_sm")
spacy_fr_sm.add_pipe("language_detector")

In [24]:
reconstructed = []
section_idx2section_name = {}
prev_meta = test_par[0].split("\t")[0]
current_txt = ""
reconstructed_idx = 0
for fragment in test_par:
    meta = fragment.split("\t")[0]
    text = fragment.split("\t")[2]
    if meta == prev_meta:
        # still in same section, check lang before adding text
        if spacy_fr_sm(text)._.language == "fr":
            current_txt += " "+text
    else:
        # started a new section, append current_txt
        reconstructed.append(current_txt)
        # add metadata to dict
        section_idx2section_name[reconstructed_idx] = meta
        # check lang then re-initialize current_txt
        if spacy_fr_sm(text)._.language == "fr":
            current_txt = text
        # re-initialize reconstructed_idx regardless of lang
        reconstructed_idx += 1
        # re-initialize prev_meta
        prev_meta = meta
# add last section to reconstructed
reconstructed.append(current_txt)
section_idx2section_name[reconstructed_idx] = meta

In [25]:
len(section_idx2section_name)

44

In [26]:
len(reconstructed)

44

In [28]:
section_idx2section_name[43]

'book=3,chapter=3,section=26'

In [29]:
reconstructed[-1]

"Souveraine Athéné, par l’ordre d’Apollon\n J’arrive : au malheureux que ton accueil soit bon!\n Ce n’est point teint de sang qu’ici je me présente :\n Sur ma main a séché la tache pâlissante;\n Tandis que s’effaçaient des souvenirs vieillis,\n J’ai d’asile en asile erré par tout pays.\n Du continent, des mers j’ai franchi l’étendue,\n Envoyé par la voix dans Delphes entendue.\n Au pied de ton image, en ton temple clément\n Je viens m’asseoir; et là j’attends le jugement.\n Bien! c’est de son passage un évident indice;\n Suis la trace de sang, muette délatrice.\n Comme le chien dépiste un faon qui fuit blessé,\n A ces gouttes je vois qu’il est par là passé.\n Un souffle haletant fatigue ma poitrine;\n Car par toute la terre à courir je m’obstine;\n Et sur mer, sans avoir les ailes d’un oiseau,\n J’ai volé, j’ai suivi son rapide vaisseau.\n Mais il est par ici caché, je le devine;\n L’odeur du sang humain a flatté ma narine.\n Cherchons, cherchons de toutes parts :\n Que rien n’échappe 

# check embeds from runner.py test runs

In [1]:
import numpy as np
import pandas as pd
from functions import load_txt_as_lst

In [2]:
prefix = "/scratch/craig.car/src_data/urn:cts:greekLit:tlg0004.tlg001"
emb_path = prefix+".emb"
dim = 768
embeds = np.fromfile(emb_path, dtype=np.float32, count=-1)
embeds.resize(embeds.shape[0] // dim, dim)
embeds.shape

(64241, 768)

In [3]:
txt_path = prefix+".overlaps"


In [4]:
txt = load_txt_as_lst(txt_path)


In [5]:
len(txt)

64241